# Conda Enviroment should be enabled before you run the next cell!

In [ ]:
# Install the requirements for this notebook
%pip install -r requirements.txt

In [ ]:
# Ensures local repository updates are reflected by colab
%load_ext autoreload
%autoreload

## Imports and Config

In [ ]:
import os
import json
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             fbeta_score, roc_auc_score, average_precision_score, confusion_matrix)
from sklearn.utils.class_weight import compute_class_weight

import xgboost as xgb
from catboost import CatBoostClassifier


%matplotlib inline

class Config:
    def __init__(self):
        # General configuration
        self.seed = 239081663        

        # Data paths
        self.data_dir = "./Data"
        self.tabular_data_path = f"{self.data_dir}/Cleaned Covid Data.csv" 
        self.prompt_data_path = f"{self.data_dir}/Cleaned Prompt Covid Data.csv" 

        # Data splits
        self.train_split = 0.7  # 70% for training
        self.test_split = 1 - self.train_split  # 30% for testing
        self.gender_split_type = "GenderUnbalanced" # Options:'AllMale', 'AllFemale', 'AllFemalePregnant', 'GenderBalanced', 'GenderUnbalanced' 

        self.target_col = 'PNEUMONIA' 
        ''  # Adjust based on your dataset
        self.columns_to_drop = [col for col in ["COVID-19 PRESENCE", "COVID-19 SEVERITY", "DEATH", "PNEUMONIA"] if col != self.target_col]
        self.num_classes = 4 if self.target_col == 'COVID-19 SEVERITY' else 2 

        # Parameter grids
        self.xgb_param_grid = {
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1],
            'n_estimators': [100, 200],  # n_estimators corresponds to epochs
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'gamma': [0, 0.1],
        }

        self.cat_param_grid = {
            'depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1],
            'iterations': [100, 200],  # iterations corresponds to epochs
            'bootstrap_type': ['Bernoulli'],
            'subsample': [0.8, 1.0],
            'colsample_bylevel': [0.8, 1.0],
            'l2_leaf_reg': [1, 3, 5],
        }

cfg = Config()


## Data Splitting

In [ ]:
def split_data_dual(df_tab, df_prompt, dataset_type="Original", test_size=0.2, random_state=42):
    """
    Splits the original tabular DataFrame and its associated prompt DataFrame into training and test sets.
    The split is done on df_tab (which is assumed to have the same index as df_prompt) and then the same
    indices are used to extract rows from df_prompt.

    The test set is created to be balanced (50% male, 50% female) by performing stratified splitting on the SEX column.
    The training set is then filtered based on the dataset_type:
      - "Original": use the full training set.
      - "Gender-Balance": sample equal numbers of males and females.
      - "Male-Only": keep only male data points.
      - "Female-Only": keep only female data points.
      - "Non-pregnant Females": from females, select rows where PREGNANT == 2.
      - "Pregnant Females": from females, select rows where PREGNANT == 1.

    Returns:
      A tuple of tuples:
        ((train_tab, train_prompt), (test_tab, test_prompt))
    """
    # Split df_tab by gender for stratification.
    male_data = df_tab[df_tab['SEX'] == 2]
    female_data = df_tab[df_tab['SEX'] == 1]
    
    # Determine test fraction for each gender. Since overall test_size is for the entire dataset,
    # and each gender should contribute equally, we use test_size/0.5 for each group.
    male_train, male_test = train_test_split(male_data, test_size=test_size/0.5, random_state=random_state)
    female_train, female_test = train_test_split(female_data, test_size=test_size/0.5, random_state=random_state)
    
    # Combine the test sets to form a balanced test set.
    test_df_tab = pd.concat([male_test, female_test])
    
    # Combine remaining data for training.
    train_df_tab = pd.concat([male_train, female_train])
    
    # Filter the training set based on dataset_type.
    if dataset_type == "GenderUnbalanced":
        filtered_train_tab = train_df_tab.copy()
    elif dataset_type == "GenderBalanced":
        min_count = min(len(male_train), len(female_train))
        filtered_males = male_train.sample(n=min_count, random_state=random_state)
        filtered_females = female_train.sample(n=min_count, random_state=random_state)
        filtered_train_tab = pd.concat([filtered_males, filtered_females])
    elif dataset_type == "AllMale":
        filtered_train_tab = train_df_tab[train_df_tab['SEX'] == 2]
    elif dataset_type == "AllFemale":
        filtered_train_tab = train_df_tab[train_df_tab['SEX'] == 1]
    elif dataset_type == "AllFemalePregnant":
        filtered_train_tab = train_df_tab[(train_df_tab['SEX'] == 1) & (train_df_tab['PREGNANT'] == 1)]
    else:
        raise ValueError("Invalid dataset_type provided. Choose from: 'Original', 'Gender-Balance', 'Male-Only', 'Female-Only', 'Non-pregnant Females', 'Pregnant Females'.")
    
    # Get the indices for training and test sets.
    train_idx = filtered_train_tab.index
    test_idx = test_df_tab.index

    # Select the corresponding rows from the prompt DataFrame.
    train_df_prompt = df_prompt.loc[train_idx]
    test_df_prompt = df_prompt.loc[test_idx]
    
    return (filtered_train_tab, train_df_prompt), (test_df_tab, test_df_prompt)

# Example usage:
# Assume df_tabular is your tabular data and df_prompts is the corresponding prompt dataframe.
df_tabular = pd.read_csv(cfg.tabular_data_path)
df_prompts = pd.read_csv(cfg.prompt_data_path)
filtered = df_tabular[(df_tabular['SEX'] == 1) & (df_tabular['PREGNANT'] == 1)]


# For example, to split using the "Pregnant Females" subset for training:
((train_tab, train_prompt), (test_tab, test_prompt)) = split_data_dual( df_tabular, df_prompts, dataset_type=cfg.gender_split_type, test_size=0.2, random_state=cfg.seed)


In [ ]:
train_tab

### Dropping non-feature columns and pulling target column

In [ ]:
#DROP CODING COLUMNS    
tab_train_data = train_tab.drop(columns=cfg.columns_to_drop, errors='ignore')
tab_test_data = test_tab.drop(columns=cfg.columns_to_drop, errors='ignore')

prompt_train_data = train_prompt.drop(columns=cfg.columns_to_drop, errors='ignore')
prompt_test_data = test_prompt.drop(columns=cfg.columns_to_drop, errors='ignore')

# Extract features (X) and target (y) for both datasets
tab_x_train = tab_train_data.loc[:, 'AGE':'USMER']
tab_y_train = tab_train_data[cfg.target_col]



tab_x_test = tab_train_data.loc[:, 'AGE':'USMER']
tab_y_test = tab_train_data[cfg.target_col]

if cfg.target_col != "COVID-19 SEVERITY":
    tab_y_train = tab_y_train - 1 
    tab_y_test = tab_y_test - 1 

In [ ]:
# Display all column names
print(tab_train_data.columns.tolist())

In [ ]:
tab_x_train, tab_y_train

In [ ]:
tab_x_test, tab_y_test

## Handling Class Imbalance

In [ ]:
# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(tab_y_train), y=tab_y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# Display class distribution in training set
class_counts = tab_y_train.value_counts().sort_index()
class_distribution = class_counts / class_counts.sum()
print("Class Distribution in Training Set:")
print(class_distribution)

### Cross-Validation

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=cfg.seed)
scoring = 'f1' if cfg.num_classes == 2 else 'f1_macro'

## Logging Function

In [ ]:
def log_to_json(file_type, model_name, gender_split_type, target_col, message):
    """
    Logs a message to a JSON file, converting NumPy data types to native Python types.

    Args:
        message (dict or str): The message or metrics to log.
        json_path (str): Path to the JSON log file.
        log_type (str): Type of log entry (e.g., 'info', 'metrics', 'error').
    """
    # Helper function to convert NumPy types to native Python types
    def convert_numpy(obj):
        if isinstance(obj, dict):
            return {k: convert_numpy(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy(elem) for elem in obj]
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.bool_):
            return bool(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj
        
    json_filename = f"{model_name}_{gender_split_type}_{target_col}.json"
    if file_type == "Feature Importance":
        json_path = os.path.join('Feature Importance Output Files/CML', json_filename)
    if file_type == "Metric":
        json_path = os.path.join('Metric Output Files/CML', json_filename)

    os.makedirs(os.path.dirname(json_path), exist_ok=True)

    # Convert and write the message (always overwrite)
    with open(json_path, 'w') as json_file:
        json.dump(convert_numpy(message), json_file, indent=4)


## Model Training and Evaluation

### XGBoost Model Training

In [ ]:
# Map n_estimators to epochs for logging
epochs = cfg.xgb_param_grid['n_estimators']

print("Starting XGBoost training...")

# Create an instance of the XGBClassifier
xgb_model = xgb.XGBClassifier(random_state=cfg.seed, use_label_encoder=False, eval_metric='logloss')

# Use GridSearchCV for hyperparameter tuning
grid_search_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=cfg.xgb_param_grid,
    scoring=scoring,
    cv=cv,
    n_jobs=-1,
    verbose=1
)


grid_search_xgb.fit(tab_x_train, tab_y_train)
print(" XGBoost training complete")

best_xgb_model = grid_search_xgb.best_estimator_

### CatBoost Model Training

In [ ]:
# # Map iterations to epochs for logging
# epochs = f"{cfg.cat_param_grid['iterations'][0]}-{cfg.cat_param_grid['iterations'][-1]}"
# # Initialize CatBoost model
# cat_model = CatBoostClassifier(
#     loss_function='Logloss' if cfg.num_classes == 2 else 'MultiClass',
#     eval_metric='F1' if cfg.num_classes == 2 else 'TotalF1',
#     class_weights=class_weights.tolist(),
#     random_seed=cfg.seed,
#     verbose=1
# )

# # Set up GridSearchCV for CatBoost using parameter grid from config
# grid_search_cat = GridSearchCV(
#     estimator=cat_model,
#     param_grid=cfg.cat_param_grid,
#     scoring=scoring,
#     cv=cv,
#     verbose=1,
#     n_jobs=-1,
#     return_train_score=True  # Enable capturing training scores
# )

# # Fit GridSearchCV
# print("Starting CatBoost training...")
# grid_search_cat.fit(tab_x_train, tab_y_train)
# best_cat_model = grid_search_cat.best_estimator_
# print("CatBoost training completed.")


## Evaluation Metrics

In [ ]:
def specificity_score(tn, fp):
    return tn / (tn + fp) if (tn + fp) > 0 else 0

def safe_roc_auc_score(y_true, y_pred, average=None, multi_class=None):
    # Check if there is more than one class in y_true
    if len(set(y_true)) == 1:
        print("Only one class present in y_true. ROC AUC score is not defined.")
        return 0  # Or return a custom value like 0.5 if you prefer
    else:
        if average:
            return roc_auc_score(y_true, y_pred, average=average, multi_class=multi_class)
        else:
            return roc_auc_score(y_true, y_pred)


def compute_specificity_macro_weighted(metrics, y_true, num_classes, average='macro'):
    specificity_per_class = []
    support = []

    for i in range(num_classes):
        tn = metrics[f'class_{i}_tn']
        fp = metrics[f'class_{i}_fp']
        specificity = specificity_score(tn, fp)
        specificity_per_class.append(specificity)

        # Support is the number of true instances for the class
        support.append(np.sum(y_true == i))

    if average == 'macro':
        # Macro average: simple mean of the per-class specificity scores
        return np.mean(specificity_per_class)
    elif average == 'weighted':
        # Weighted average: weighted mean, weighting by the support of each class
        return np.average(specificity_per_class, weights=support)
    else:
        raise ValueError("Unsupported average type. Use 'macro' or 'weighted'.")

def compute_metrics(y_true, y_pred, y_prob, average='binary', num_classes=2):
    try:
        # Ensure y_true and y_pred are flattened
        y_true = np.concatenate(y_true).ravel()  # Flatten y_true
        y_pred = np.concatenate(y_pred).ravel()  # Flatten y_pred
    except:
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

    metrics = {}
    # Compute overall specificity (macro or weighted)
    if num_classes > 2:
        # Calculate TP, FP, TN, FN for each class in the multi-class setting
        for i in range(num_classes):
            metrics[f'class_{i}_tp'] = np.sum((y_true == i) & (y_pred == i))
            metrics[f'class_{i}_fp'] = np.sum((y_true != i) & (y_pred == i))
            metrics[f'class_{i}_fn'] = np.sum((y_true == i) & (y_pred != i))
            metrics[f'class_{i}_tn'] = np.sum((y_true != i) & (y_pred != i))

        metrics['specificity_macro'] = compute_specificity_macro_weighted(metrics, y_true, num_classes, average='macro')
        metrics['specificity_weighted'] = compute_specificity_macro_weighted(metrics, y_true, num_classes, average='weighted')
    else:  # Binary case
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        metrics['tp'] = tp
        metrics['tn'] = tn
        metrics['fp'] = fp
        metrics['fn'] = fn
        metrics['specificity'] = specificity_score(tn=metrics['tn'], fp=metrics['fp'])

    # General metrics for both binary and multi-class cases
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['precision'] = precision_score(y_true, y_pred, average=average)
    metrics['recall'] = recall_score(y_true, y_pred, average=average)
    metrics['f1'] = f1_score(y_true, y_pred, average=average)
    metrics['f2'] = fbeta_score(y_true, y_pred, beta=2, average=average)

    # AUC & Average Precision: Adapted for multi-class scenarios
    if average != 'binary':  # Multi-class scenario
        metrics['roc_auc'] = safe_roc_auc_score(y_true, y_prob, average=average, multi_class='ovr')
        metrics['au_prc'] = average_precision_score(y_true, y_prob, average=average)
    else:  # Binary case
        # y_prob = np.concatenate(y_prob).ravel()
        y_prob = y_prob[:, 1]
        metrics['roc_auc'] = safe_roc_auc_score(y_true, y_prob)
        metrics['au_prc'] = average_precision_score(y_true, y_prob)

    # Sensitivity is the same as recall in both cases
    metrics['sensitivity'] = metrics['recall']

    return metrics

def print_metrics(metrics, num_classes):
    print("Metrics Summary:")
    print("---------------------------------------------------")
    if num_classes > 2:
        # Calculate TP, FP, TN, FN for each class in the multi-class setting
        for i in range(num_classes):
            print(f"True Positives (TP) for class {i}:      {metrics[f'class_{i}_tp']}")
            print(f"True Negatives (TN) for class {i}:      {metrics[f'class_{i}_tn']}")
            print(f"False Positives (FP) for class {i}:     {metrics[f'class_{i}_fp']}")
            print(f"False Negatives (FN) for class {i}:     {metrics[f'class_{i}_fn']}")
        print(f"Macro Specificity:                      {metrics['specificity_macro']:.4f}")
        print(f"Weighted Specificity:                   {metrics['specificity_weighted']:.4f}")
    else:
        print(f"True Positives (TP):     {metrics['tp']}")
        print(f"True Negatives (TN):     {metrics['tn']}")
        print(f"False Positives (FP):    {metrics['fp']}")
        print(f"False Negatives (FN):    {metrics['fn']}")
        print(f"Specificity:             {metrics['specificity']:.4f}")

    print(f"Accuracy:                {metrics['accuracy']:.4f}")
    print(f"Precision:               {metrics['precision']:.4f}")
    print(f"Recall:                  {metrics['recall']:.4f}")
    print(f"F1 Score:                {metrics['f1']:.4f}")
    print(f"F2 Score:                {metrics['f2']:.4f}")
    print(f"ROC AUC:                 {metrics['roc_auc']:.4f}")
    print(f"Area Under PR Curve:     {metrics['au_prc']:.4f}")
    print(f"Sensitivity:             {metrics['sensitivity']:.4f}")
    print("---------------------------------------------------")



In [ ]:
def evaluate_model(model, X, y, model_name, dataset_name, cfg):
    y_pred = model.predict(X)
    if cfg.num_classes == 2:
        y_pred_prob = model.predict_proba(X)  # Shape (n_samples, 2)
    else:
        y_pred_prob = model.predict_proba(X)

    # Compute metrics using compute_metrics function
    average = 'binary' if cfg.num_classes == 2 else 'macro'
    metrics = compute_metrics(y, y_pred, y_pred_prob, average=average, num_classes=cfg.num_classes)

    # Print metrics using print_metrics function
    print(f"--- {model_name} Metrics on {dataset_name} Set ---")
    print_metrics(metrics, cfg.num_classes)

    # Plot confusion matrix
    cm = confusion_matrix(y, y_pred)
    metrics['confusion_matrix'] = cm.tolist()  # Ensure it's serializable

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=np.unique(y),
                yticklabels=np.unique(y))
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'{model_name} Confusion Matrix ({dataset_name} Set)')
    plt.show()

    # Add dataset and model name to metrics
    metrics.update({
        'dataset': dataset_name,
        'model_name': model_name,
    })
    return metrics

### XGBoost Evaluation

In [ ]:
# Evaluate the model on different datasets
metrics_train_xgb = evaluate_model(best_xgb_model, tab_x_train, tab_y_train, model_name='XGBoost',dataset_name='Training', cfg=cfg)
metrics_test_xgb = evaluate_model(best_xgb_model, tab_x_test, tab_y_test, model_name='XGBoost', dataset_name='Test', cfg=cfg)

log_to_json(file_type = "Metric",
            model_name='Xgboost',
            gender_split_type = cfg.gender_split_type, 
            target_col = ''.join(word.capitalize() for word in cfg.target_col.split()), 
            message=metrics_test_xgb
)

### CatBoost Evaluation

In [ ]:
# # Evaluate CatBoost model on training set
# metrics_train_cat = evaluate_model(best_cat_model, tab_x_train, tab_y_train, model_name='CatBoost', dataset_name='Training', cfg=cfg)


# # Evaluate CatBoost model on test set
# metrics_test_cat = evaluate_model(best_cat_model, tab_x_test, tab_y_test, model_name='CatBoost', dataset_name='Test', cfg=cfg)

# # Log metrics
# log_to_json(file_type = "Metric",
#             model_name='Catboost',
#             gender_split_type = cfg.gender_split_type, 
#             target_col =  ''.join(word.capitalize() for word in cfg.target_col.split()), 
#             message=metrics_test_cat
#             )


## Feature Importance and Plots

In [ ]:
def format_feature_importance(raw_importance):
    """
    Converts a list of feature importances into a formatted dictionary with
    proper capitalization, spacing, and rounded values.

    Args:
        raw_importance (list): List of dictionaries with 'Feature' and 'Importance' keys.

    Returns:
        dict: Formatted feature importance dictionary.
    """
    name_map = {
        "AGE": "Age",
        "MEDICAL_UNIT": "Medical unit",
        "USMER": "Usmr",
        "ICU": "ICU",
        "OBESITY": "Obesity",
        "INTUBED": "Intubed",
        "OTHER_DISEASE": "Other disease",
        "HYPERTENSION": "Hypertension",
        "DIABETES": "Diabetes",
        "PATIENT_TYPE": "Patient Type",
        "ASTHMA": "Asthma",
        "TOBACCO": "Tobacco",
        "INMSUPR": "Inmsupr",
        "COPD": "COPD",
        "CARDIOVASCULAR": "Cardiovascular",
        "RENAL_CHRONIC": "Renal chronic",
        "PREGNANT": "Pregnancy",
        "SEX": "Sex"
    }

    return {
        name_map.get(item['Feature'], item['Feature'].title().replace("_", " ")): round(item['Importance'], 2)
        for item in raw_importance
    }


### XGBoost Feature Importance

In [ ]:
# Compute and log feature importance
num_top_features = 20

# Feature importance
importance = best_xgb_model.get_booster().get_score(importance_type='weight')

importance_df = pd.DataFrame({
    'Feature': list(importance.keys()),
    'Importance': list(importance.values())
})

# Ensure all features from the dataset are included
all_features = list(tab_x_train.columns)  # Get all feature names from the dataset

# Create a DataFrame with all features, setting missing ones to 0 importance
full_importance_df = pd.DataFrame({'Feature': all_features})

# Merge with existing importance data, filling NaN values with 0
full_importance_df = full_importance_df.merge(importance_df, on='Feature', how='left').fillna(0)

full_importance_df['Importance'] /= full_importance_df['Importance'].sum()
full_importance_df = full_importance_df.sort_values(by='Importance', ascending=False)[:num_top_features]

# Plot the feature importance
plt.figure(figsize=(10, 6))
plt.barh(full_importance_df['Feature'], full_importance_df['Importance'])
plt.title('XGBoost Feature Importance')
plt.xlabel('Importance')
plt.gca().invert_yaxis()
plt.show()

# Log feature importance
raw_importance_dict = full_importance_df.to_dict('records')
feature_importance_dict = format_feature_importance(raw_importance_dict)
log_to_json(file_type = "Feature Importance",
            model_name='Xgboost',
            gender_split_type = cfg.gender_split_type, 
            target_col = ''.join(word.capitalize() for word in cfg.target_col.split()), 
            message=feature_importance_dict
            )


### CatBoost Feature Importance

In [ ]:
# # Feature importance
# importance = best_cat_model.get_feature_importance()
# # Ensure that the length of importance matches the number of features
# if len(importance) != tab_x_train.shape[1]:
#     raise ValueError("Mismatch between number of features and feature importance scores.")

# importance_df = pd.DataFrame({
#     'Feature': tab_x_train.columns,
#     'Importance': importance
# })
# importance_df['Importance'] /= importance_df['Importance'].sum()
# importance_df = importance_df.sort_values(by='Importance', ascending=False)

# # Plot the feature importance
# plt.figure(figsize=(10, 6))
# plt.barh(importance_df['Feature'], importance_df['Importance'])
# plt.title('CatBoost Feature Importance')
# plt.xlabel('Importance')
# plt.gca().invert_yaxis()
# plt.show()

# # Log feature importance
# raw_importance_dict = full_importance_df.to_dict('records')
# feature_importance_dict = format_feature_importance(raw_importance_dict)
# log_to_json(file_type = "Feature Importance",
#             model_name='Catboost',
#             gender_split_type = cfg.gender_split_type, 
#             target_col = ''.join(word.capitalize() for word in cfg.target_col.split()), 
#             message=feature_importance_dict
#             )